# Feature Engineering의 목표

- 사고 단위 기준의 모델링을 위해 1:N 관계로 구성된 원천 데이터셋을 AccidentId 기준의 1:1 집계 데이터로 변환한다
- 생성된 집계 피처들을 Accidents 데이터셋을 기준으로 병합하여 사고 단위의 학습 데이터를 구성한다.
- 모든 데이터 병합 과정에서 AccidentId를 공통 매핑 키로 사용한다.

In [1]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np

# 1. 집계 데이터 생성하기

## 1) Users 집계 데이터 생성하기

### Users 집계(users_agg) 생성 기준

* Users 데이터는 Seat·Gender·Age·Safety 등 인적 정보의 품질이 상대적으로 안정적이어서 사고 단위 피처 생성을 위한 기준 테이블로 사용하였다.
* Seat 정보를 활용하여 운전자·동승자·보행자를 구분함으로써 사고 참여 인원 구조를 사고 단위로 명확히 집계할 수 있다.
* 안전벨트 착용 여부, 성별, 연령대 정보는 교통사고 분석 분야에서 사고 심각도와의 연관성이 논의되는 변수들이며
* 본 프로젝트에서는 해당 요인들이 사고 결과에 미치는 영향을 모델 학습을 통해 확인하고자 집계 피처로 포함하였다.
* 운전자 나이(평균·최소·최대)는 사고에 포함된 운전자 집단의 연령 분포를 요약하기 위한 통계적 지표로 생성하였다.
* 연령 계산은 사고 발생 시점을 기준으로 하기 위해 2018년을 기준 연도로 사용하였다.
* Category 정보는 차량 테이블의 Category보다 사용자 정보 기반 Category가 품질이 우수하다고 판단하여 이를 사용하였다.

### Users 집계 변수

- 사고번호 : AccidentId
- 사고에 참여한 차량 수 : Vehicle_count_user
- 차량종류 : 종류에 따라 컬럼생성 (ex: Category_차량라벨)
- 사고에 참여한 전체 인원 : Persons
- 운전자 인원 : Drivers
- 동승자 인원 : Passengers
- 좌석 미기록 : SeatNan
- 보행자 수 : Pedestrian
- 안전밸트를 사용한 인원 수 : Safety_used_yes_count
- 안전밸트를 사용하지 않은 인원 수 : Safety_used_no_count
- 안전밸트 사용여부 확인되지 않은 인원 수 : Safety_used_unknown_count
- 안전밸트 사용여부 누락된 인원 수 : Safety_used_missing_count
- 남성의 수 : Male_count
- 여성의 수 : Female_count
- 성별을 알 수 없는 인원 : Gender_unknown
- 운전자의 평균 나이 : Driver_age_mean
- 운전자의 최소 나이 : Driver_age_min
- 운전자의 최대 나이 : Driver_age_max
- 동승자_유아수 : Passenger_child (0-8)
- 동승자_청소년수 : Passenger_youth(9-19)
- 동승자_청년수 : Passenger_adult(20-34)
- 동승자_중장년수 : Passenger_senior(35-65)
- 동승자_노년수 : Passenger_elder(65+)
- 운전자 나이 미상 여부 : Driver_age_unknown
- 동승자 나이 미상 여부 : Passenger_age_unknown

In [2]:
# 데이터 dtype() 지정하기
dtype_dict = {
    "AccidentId": "int64",
    "VehicleId": "string",
    "Category": "string",
    "Gender": "string",
    "TripReason": "string",
    "SafetyDevice": "string",
    "SafetyDeviceUsed": "string",
    "PedestrianLocation": "string",
    "PedestrianAction": "string",
    "PedestrianCompany": "string",
    "BirthYear": "Int64",
    "Seat": "Int64"
}

In [3]:
# 데이터 불러오기
users_test = pd.read_csv('./data/Cleaning/users_test_Cleaning.csv', dtype=dtype_dict)
users_train = pd.read_csv('./data/Cleaning/users_train_Cleaning.csv', dtype=dtype_dict)

In [4]:
users_train

,AccidentId,VehicleId,Seat,Category,Gender,TripReason,SafetyDevice,SafetyDeviceUsed,PedestrianLocation,PedestrianAction,PedestrianCompany,BirthYear
0,201800000001,A01,1,Car<=3.5T,Male,Leisure,SeatBelt,Yes,PedestrianNaN,PedestrianNaN,Unknown,1960
1,201800000001,B01,1,Car<=3.5T,Male,Unknown,SeatBelt,Yes,PedestrianNaN,PedestrianNaN,Unknown,1928
2,201800000002,A01,1,Car<=3.5T,Male,Unknown,SeatBelt,Yes,PedestrianNaN,PedestrianNaN,Unknown,1947
3,201800000002,A01,99,Car<=3.5T,Male,Unknown,Helmet,Missing,OnLane<=OnSidewalk0mCrossing,Crossing,Alone,1965
4,201800000003,A01,1,Motorbike>125cm3,Male,Leisure,Helmet,Yes,PedestrianNaN,PedestrianNaN,Unknown,1937
...,...,...,...,...,...,...,...,...,...,...,...,...
118377,201800057781,A01,99,Car<=3.5T,Male,Other,ChildrenDevice,Missing,OnCrossingWithoutLights,Crossing,Alone,1975
118378,201800057782,A01,1,Car<=3.5T,Female,Other,SeatBelt,Yes,PedestrianNaN,PedestrianNaN,PedestrianNaN,1993
118379,201800057782,B01,1,Motorbike_50cm3-125cm3,Male,Other,Helmet,Yes,PedestrianNaN,PedestrianNaN,PedestrianNaN,1974
118380,201800057783,A01,1,Utility,Female,Other,SeatBelt,Yes,PedestrianNaN,PedestrianNaN,PedestrianNaN,1934


- Seat는 현재 좌석 위치를 나타내는 정수형 변수로 구성되어 있으며 해당 값을 그대로 활용해 집계할 경우 불필요한 희소 행렬이 생성될 가능성이 높다.
- Seat 값은 좌석 번호 자체로 사용하지 않고 운전자는 Driver, 동승자는 Passenger, 보행자는 pedestrian으로 재분류하여 집계에 활용한다.

In [5]:
# Seat 변수 값 숫자에서 문자열로 수정하기

# 매핑키 만들기
seat_map = {
    1.0: "Driver",
    2.0: "Passenger",
    3.0: "Passenger",
    4.0: "Passenger",
    5.0: "Passenger",
    6.0: "Passenger",
    7.0: "Passenger",
    8.0: "Passenger",
    9.0: "Passenger",
    99: "Pedestrian",
    -999: "Unknown"
}

users_test["Seat"] = users_test["Seat"].map(seat_map)
users_train["Seat"] = users_train["Seat"].map(seat_map)

In [6]:
# 변경된 라벨 확인
print(f'test 표본 수 : \n{users_test["Seat"].value_counts()}\n')
print(f'train 표본 수 : \n{users_train["Seat"].value_counts()}')

test 표본 수 : 
Seat
Driver        9614
Passenger     2410
Pedestrian    1128
Unknown        220
Name: count, dtype: int64

train 표본 수 : 
Seat
Driver        86385
Passenger     20607
Pedestrian     9330
Unknown        2060
Name: count, dtype: int64


- users 데이터의 출생년도는 사고 발생 시점을 기준으로 하기 위해 2018년을 기준 연도로 설정하여 나이(Age)로 변환한다.
- 변환된 나이 변수는 해석 용이성과 집계를 고려하여 연령대 그룹으로 범주화한다.

In [7]:
# 출생년도로 나이 컬럼 생성하기
users_test["Age"] = 2018 - users_test["BirthYear"]
users_train["Age"] = 2018 - users_train["BirthYear"]

In [8]:
# 나이 그룹 함수 생성
def categorize_age(age):
    
    if age == 3017 :        # -999가 나이로 변환되면서 3017로 변환됨
        return "Unknown"
        
    age = float(age)
    if age <= 8:
        return "child"
    elif age <= 19:
        return "youth"
    elif age <= 34:
        return "adult"
    elif age <= 65:
        return "senior"
    else:
        return "elder"

In [9]:
# 나이를 기준으로 그룹핑한 컬럼 생성하기
users_test["AgeGroup"] = users_test["Age"].apply(categorize_age)
users_train["AgeGroup"] = users_train["Age"].apply(categorize_age)

In [10]:
# AgeGroup 라벨 확인
print(f'test 표본 수 : \n{users_test["AgeGroup"].value_counts()}\n')
print(f'train 표본 수 : \n{users_train["AgeGroup"].value_counts()}')

test 표본 수 : 
AgeGroup
senior     5577
adult      4420
youth      1539
elder      1307
child       355
Unknown     174
Name: count, dtype: int64

train 표본 수 : 
AgeGroup
senior     43464
elder      29478
adult      28844
youth       9240
Unknown     4065
child       3291
Name: count, dtype: int64


- 원-핫 인코딩을 위한 컬럼을 준비하겠습니다.

In [11]:
# Category 원-핫 인코딩 칼럼 준비하기

# 원-핫 인코딩(0/1 더미 컬럼)으로 변환
cat_dummies = pd.get_dummies(users_test["Category"], prefix="Category")
cat_dummies_train = pd.get_dummies(users_train["Category"], prefix="Category")

# 더미 컬럼들을 옆(열 방향)으로 붙이기
users_test = pd.concat([users_test, cat_dummies], axis=1)
users_train = pd.concat([users_train, cat_dummies_train], axis=1)

#  "Category_로 시작하는 더미 컬럼 목록"을 자동으로 추출
category_cols = [c for c in users_test.columns if c.startswith("Category_")]
category_cols_train = [c for c in users_train.columns if c.startswith("Category_")]

- users_agg 집계 데이터 생성

In [12]:
# 집계 users_agg_test 만들기
users_agg_test = (
    users_test
    .groupby("AccidentId")
    .apply(lambda g: pd.Series({

        # 1. users에서 확인된 사고 차량 수
        "Vehicle_count_user": g["VehicleId"].nunique(),

        # 2. 전체 인원
        "Persons": len(g),

        # 3. 운전자 / 동승자 / 보행자 / 좌석미기록 수 (Seat 직접 사용)
        "Drivers": (g["Seat"] == "Driver").sum(),
        "Passengers": (g["Seat"] == "Passenger").sum(),
        "Pedestrian": (g["Seat"] == "Pedestrian").sum(),
        "SeatNan": (g["Seat"] == "Unknown").sum(),

        # 4. 안전벨트
        "Safety_used_yes_count":     (g["SafetyDeviceUsed"] == "Yes").sum(),
        "Safety_used_no_count":      (g["SafetyDeviceUsed"] == "No").sum(),
        "Safety_used_unknown_count": (g["SafetyDeviceUsed"] == "Unknown").sum(),
        "Safety_used_missing_count": (g["SafetyDeviceUsed"] == "Missing").sum(),

        # 5. 성별
        "Male_count":   (g["Gender"] == "Male").sum(),
        "Female_count": (g["Gender"] == "Female").sum(),
        "Gender_unknown": (g["Gender"] == "Unknown").sum(),

        # 6. 운전자 나이 통계
        "Driver_age_mean": g.loc[g["Seat"] == "Driver", "Age"].mean(),
        "Driver_age_min":  g.loc[g["Seat"] == "Driver", "Age"].min(),
        "Driver_age_max":  g.loc[g["Seat"] == "Driver", "Age"].max(),

        # 6-1. 운전자 나이 미상 여부
        "Driver_age_unknown": ((g["Seat"] == "Driver") & (g["Age"].isna())).sum(),

        # 7. 동승자 연령대별 count
        "Passenger_child":  ((g["Seat"] == "Passenger") & (g["AgeGroup"] == "child")).sum(),
        "Passenger_youth":  ((g["Seat"] == "Passenger") & (g["AgeGroup"] == "youth")).sum(),
        "Passenger_adult":  ((g["Seat"] == "Passenger") & (g["AgeGroup"] == "adult")).sum(),
        "Passenger_senior": ((g["Seat"] == "Passenger") & (g["AgeGroup"] == "senior")).sum(),
        "Passenger_elder":  ((g["Seat"] == "Passenger") & (g["AgeGroup"] == "elder")).sum(),

        # 7-1. 동승자 나이 미상 
        "Passenger_age_unknown": ((g["Seat"] == "Passenger") & (g["AgeGroup"] == "Unknown")).sum(),

        # 8. Category_* 집계
        **{col: g[col].sum() for col in category_cols}
    }))
    .reset_index()
)

C:\Users\nini\AppData\Local\Temp\ipykernel_2164\605046153.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


In [13]:
# 집계 users_agg_train 만들기
users_agg_train = (
    users_train
    .groupby("AccidentId")
    .apply(lambda g: pd.Series({

        # 1. users에서 확인된 사고 차량 수
        "Vehicle_count_user": g["VehicleId"].nunique(),

        # 2. 전체 인원
        "Persons": len(g),

        # 3. 운전자 / 동승자 / 보행자 / 좌석미기록 수 (Seat 직접 사용)
        "Drivers": (g["Seat"] == "Driver").sum(),
        "Passengers": (g["Seat"] == "Passenger").sum(),
        "Pedestrian": (g["Seat"] == "Pedestrian").sum(),
        "SeatNan": (g["Seat"] == "Unknown").sum(),

        # 4. 안전벨트
        "Safety_used_yes_count":     (g["SafetyDeviceUsed"] == "Yes").sum(),
        "Safety_used_no_count":      (g["SafetyDeviceUsed"] == "No").sum(),
        "Safety_used_unknown_count": (g["SafetyDeviceUsed"] == "Unknown").sum(),
        "Safety_used_missing_count": (g["SafetyDeviceUsed"] == "Missing").sum(),

        # 5. 성별
        "Male_count":   (g["Gender"] == "Male").sum(),
        "Female_count": (g["Gender"] == "Female").sum(),
        "Gender_unknown": (g["Gender"] == "Unknown").sum(),

        # 6. 운전자 나이 통계
        "Driver_age_mean": g.loc[g["Seat"] == "Driver", "Age"].mean(),
        "Driver_age_min":  g.loc[g["Seat"] == "Driver", "Age"].min(),
        "Driver_age_max":  g.loc[g["Seat"] == "Driver", "Age"].max(),

        # 6-1. 운전자 나이 미상 여부
        "Driver_age_unknown": ((g["Seat"] == "Driver") & (g["Age"].isna())).sum(),

        # 7. 동승자 연령대별 count
        "Passenger_child":  ((g["Seat"] == "Passenger") & (g["AgeGroup"] == "child")).sum(),
        "Passenger_youth":  ((g["Seat"] == "Passenger") & (g["AgeGroup"] == "youth")).sum(),
        "Passenger_adult":  ((g["Seat"] == "Passenger") & (g["AgeGroup"] == "adult")).sum(),
        "Passenger_senior": ((g["Seat"] == "Passenger") & (g["AgeGroup"] == "senior")).sum(),
        "Passenger_elder":  ((g["Seat"] == "Passenger") & (g["AgeGroup"] == "elder")).sum(),

        # 7-1. 동승자 나이 미상 
        "Passenger_age_unknown": ((g["Seat"] == "Passenger") & (g["AgeGroup"] == "Unknown")).sum(),

        # 8. Category_* 집계
        **{col: g[col].sum() for col in category_cols}
    }))
    .reset_index()
)

C:\Users\nini\AppData\Local\Temp\ipykernel_2164\3526380433.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


In [14]:
# Test, Train 컬럼 확인하기

print("Train에만 있는 컬럼:", set(users_agg_train.columns) - set(users_agg_test.columns))
print("Test에만 있는 컬럼:", set(users_agg_test.columns) - set(users_agg_train.columns))

Train에만 있는 컬럼: set()
Test에만 있는 컬럼: set()


## 2) dtype 정리

In [15]:
# Driver_age_mean/min/max 제외 int로 변환하기

count_cols = [
    "Vehicle_count_user",
    "Persons",
    "Drivers",
    "Passengers",
    "SeatNan",
    "Pedestrian",
    "Safety_used_yes_count",
    "Safety_used_no_count",
    "Safety_used_unknown_count",
    "Safety_used_missing_count",
    "Male_count",
    "Female_count",
    "Gender_unknown",
    "Passenger_child",
    "Passenger_youth",
    "Passenger_adult",
    "Passenger_senior",
    "Passenger_elder",
    "Driver_age_unknown",
    "Passenger_age_unknown"
]

# Category_* 컬럼 추가
count_cols += [c for c in users_agg_test.columns if c.startswith("Category_")]

# 변환
users_agg_test[count_cols] = users_agg_test[count_cols].astype("int64")
users_agg_train[count_cols] = users_agg_train[count_cols].astype("int64")

# Driver_age_mean, Driver_age_min, Driver_age_max float64로 변환
user_agg_float_cols = ["Driver_age_mean", "Driver_age_min", "Driver_age_max"]

# Drivaer관련 변수 결측치 -999로 변환 (Drivaer가 없는 경우가 존재해 NaN 발생)
users_agg_test[user_agg_float_cols] = users_agg_test[user_agg_float_cols].fillna(-999)
users_agg_train[user_agg_float_cols] = users_agg_train[user_agg_float_cols].fillna(-999)

# 변환
users_agg_test[user_agg_float_cols] = users_agg_test[user_agg_float_cols].astype("float64")
users_agg_train[user_agg_float_cols] = users_agg_train[user_agg_float_cols].astype("float64")

C:\Users\nini\AppData\Local\Temp\ipykernel_2164\959536844.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  users_agg_test[user_agg_float_cols] = users_agg_test[user_agg_float_cols].fillna(-999)
C:\Users\nini\AppData\Local\Temp\ipykernel_2164\959536844.py:38: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  users_agg_train[user_agg_float_cols] = users_agg_train[user_agg_float_cols].fillna(-999)


In [16]:
# 최종 dtype 확인
print(f'test dtype : \n{users_agg_test.dtypes}\n')
print(f'train dtype : \n{users_agg_train.dtypes}')

test dtype : 
AccidentId                           int64
Vehicle_count_user                   int64
Persons                              int64
Drivers                              int64
Passengers                           int64
Pedestrian                           int64
SeatNan                              int64
Safety_used_yes_count                int64
Safety_used_no_count                 int64
Safety_used_unknown_count            int64
Safety_used_missing_count            int64
Male_count                           int64
Female_count                         int64
Gender_unknown                       int64
Driver_age_mean                    float64
Driver_age_min                     float64
Driver_age_max                     float64
Driver_age_unknown                   int64
Passenger_child                      int64
Passenger_youth                      int64
Passenger_adult                      int64
Passenger_senior                     int64
Passenger_elder                      int

## 3) Vehicles 집계 데이터 생성하기

### Vehicles 집계(vehicles_agg) 생성 기준

* Vehicles의 'VehicleId', 'Category'의 컬럼은 오염도가 높아 전처리 단계에서 삭제 처리하였다.
* 해당 정보는 품질이 더 안정적인 Users 데이터에서 활용하였다.
* 대부분의 변수는 범주형 데이터로 구성되어 있어 라벨별 발생 빈도를 반영할 수 있도록 원-핫 인코딩 후 사고 단위로 집계하였다.
* PassengerNumber 변수는 대부분의 값이 0에 집중되어 있고 일부 극단값이 존재하는 등 데이터 정의가 불명확하다고 판단하여 최종 집계 피처에서 제외하였다.

### Vehicles 집계 변수
- 사고번호 : AccidentId
- 진행방향 : 종류에 따라 컬럼생성 (ex: Direction_충돌라벨)
- 고정물충돌 : 종류에 따라 컬럼생성 (ex: Fixed_충돌라벨)
- 이동물체 충돌 : 종류에 따라 컬럼생성 (ex : Mobile_충돌라벨)
- 충격된 위치 : 종류에 따라 컬럼생성 (ex : ImpactPoint_충돌라벨)
- 사고 당시 차량 행동 : 종류에 따라 컬럼생성 (ex: Maneuver_행동라벨)

In [17]:
# 데이터 dtype() 지정하기
dtype_dict = {
    "AccidentId": "int64",
    "Direction": "string",
    "FixedObstacle": "string",
    "MobileObstacle": "string",
    "ImpactPoint": "string",
    "Maneuver": "string",
    "PassengerNumber": "Int64"
}

In [18]:
# 데이터 불러오기
vehicles_test = pd.read_csv('./data/Cleaning/vehicles_test_Cleaning.csv', dtype=dtype_dict)
vehicles_train = pd.read_csv('./data/Cleaning/vehicles_train_Cleaning.csv', dtype=dtype_dict)

In [19]:
# 원-핫 인코딩 컬럼 선택
onehot_cols = ["Direction", "FixedObstacle", "MobileObstacle", "ImpactPoint", "Maneuver"]

In [20]:
# 원-핫 인코딩 실행
veh_dummies_test = pd.get_dummies(vehicles_test[onehot_cols], prefix=onehot_cols)
veh_dummies_train = pd.get_dummies(vehicles_train[onehot_cols], prefix=onehot_cols)

In [21]:
# 원본 데이터에 더미 컬럼 붙이기
vehicles_test_ohe = pd.concat([vehicles_test[["AccidentId"]], veh_dummies_test], axis=1)
vehicles_train_ohe = pd.concat([vehicles_train[["AccidentId"]], veh_dummies_train], axis=1)

In [22]:
# AccidentId 기준 집계

vehicles_agg_test = (
    vehicles_test_ohe
    .groupby("AccidentId")
    .sum()
    .reset_index()
)

vehicles_agg_train = (
    vehicles_train_ohe
    .groupby("AccidentId")
    .sum()
    .reset_index()
)

In [23]:
# train / test 컬럼 정합성 확인
set(vehicles_agg_train.columns) - set(vehicles_agg_test.columns)
set(vehicles_agg_test.columns) - set(vehicles_agg_train.columns)

set()

In [24]:
# 최종 dtype 확인
print(f'test dtype : \n{vehicles_agg_test.dtypes}\n')
print(f'train dtype : \n{vehicles_agg_train.dtypes}')

test dtype : 
AccidentId                          int64
Direction_Decreasing                Int64
Direction_Increasing                Int64
Direction_Unknown                   Int64
FixedObstacle_BuildingWallOrPier    Int64
                                    ...  
Maneuver_SwerveToRight              Int64
Maneuver_TurnToLeft                 Int64
Maneuver_TurnToRight                Int64
Maneuver_UTurnInLane                Int64
Maneuver_WrongWay                   Int64
Length: 63, dtype: object

train dtype : 
AccidentId                          int64
Direction_Decreasing                Int64
Direction_Increasing                Int64
Direction_Unknown                   Int64
FixedObstacle_BuildingWallOrPier    Int64
                                    ...  
Maneuver_SwerveToRight              Int64
Maneuver_TurnToLeft                 Int64
Maneuver_TurnToRight                Int64
Maneuver_UTurnInLane                Int64
Maneuver_WrongWay                   Int64
Length: 63, dtype: o

# 2. 데이터 병합하기

In [25]:
# accidents, places 데이터 로드
accidents_test = pd.read_csv('./data/Cleaning/accidents_test_Cleaning.csv', dtype=dtype_dict)
accidents_train = pd.read_csv('./data/Cleaning/accidents_train_Cleaning.csv', dtype=dtype_dict)

places_test = pd.read_csv('./data/Cleaning/places_test_Cleaning.csv', dtype=dtype_dict)
places_train = pd.read_csv('./data/Cleaning/places_train_Cleaning.csv', dtype=dtype_dict)

## 1) 구조 확인하기

In [26]:
# AccidentId 기준 1:1 구조인지 확인 함수 만들기

def check_unique(df, name):
    total = len(df)
    unique = df['AccidentId'].nunique()
    dup = total - unique
    
    print(f"\n[{name}]")
    print(f"- 전체 행 수: {total}")
    print(f"- AccidentId 고유값: {unique}")
    print(f"- 중복 AccidentId 개수: {dup}")
    print("=", "1:1 구조입니다." if dup == 0 else "AccidentId 중복 존재")

In [27]:
# Test

datasets_test = [
    (accidents_test, "Accidents"),
    (places_test, "Places"),
    (users_agg_test, "Users_agg"),
    (vehicles_agg_test, "Vehicles_agg")
]

for df, name in datasets_test:
    check_unique(df, name)


[Accidents]
- 전체 행 수: 5779
- AccidentId 고유값: 5779
- 중복 AccidentId 개수: 0
= 1:1 구조입니다.

[Places]
- 전체 행 수: 5779
- AccidentId 고유값: 5779
- 중복 AccidentId 개수: 0
= 1:1 구조입니다.

[Users_agg]
- 전체 행 수: 5779
- AccidentId 고유값: 5779
- 중복 AccidentId 개수: 0
= 1:1 구조입니다.

[Vehicles_agg]
- 전체 행 수: 5779
- AccidentId 고유값: 5779
- 중복 AccidentId 개수: 0
= 1:1 구조입니다.


In [28]:
# Train

datasets = [
    (accidents_train, "Accidents"),
    (places_train, "Places") ,
    (users_agg_train, "Users_agg"),
    (vehicles_agg_train, "Vehicles_agg")
]

for df, name in datasets:
    check_unique(df, name)


[Accidents]
- 전체 행 수: 47816
- AccidentId 고유값: 47816
- 중복 AccidentId 개수: 0
= 1:1 구조입니다.

[Places]
- 전체 행 수: 52004
- AccidentId 고유값: 52004
- 중복 AccidentId 개수: 0
= 1:1 구조입니다.

[Users_agg]
- 전체 행 수: 52002
- AccidentId 고유값: 52002
- 중복 AccidentId 개수: 0
= 1:1 구조입니다.

[Vehicles_agg]
- 전체 행 수: 52004
- AccidentId 고유값: 52004
- 중복 AccidentId 개수: 0
= 1:1 구조입니다.


## 2) merge

In [30]:
# 1) Accidents + Places
test_final = accidents_test.merge(places_test, on="AccidentId", how="left")
train_final = accidents_train.merge(places_train, on="AccidentId", how="left")

# 2) + Users_agg
test_final = test_final.merge(users_agg_test, on="AccidentId", how="left")
train_final = train_final.merge(users_agg_train, on="AccidentId", how="left")

# 3) + Vehicles_agg
test_final = test_final.merge(vehicles_agg_test, on="AccidentId", how="left")
train_final = train_final.merge(vehicles_agg_train, on="AccidentId", how="left")

In [31]:
# Test 데이터 셋 확인
test_final.head(3)

,AccidentId,Date,Hour,Light,Department,Commune,InAgglomeration,IntersectionType,Weather,CollisionType,...,Maneuver_PassRight,Maneuver_Reverse,Maneuver_SameDirectionOrLane,Maneuver_Stopped,Maneuver_SwerveToLeft,Maneuver_SwerveToRight,Maneuver_TurnToLeft,Maneuver_TurnToRight,Maneuver_UTurnInLane,Maneuver_WrongWay
0,201800000005,2018-06-26,0 days 16:05:00,Daylight,590,477,Yes,NoIntersection,Normal,2Vehicles-Side,...,0,0,0,0,0,0,0,0,0,0
1,201800000007,2018-09-26,0 days 00:40:00,NightStreelightsOn,590,133,Yes,NoIntersection,Normal,Other,...,0,0,0,0,0,0,0,0,0,0
2,201800000035,2018-03-16,0 days 21:15:00,NightNoStreetLight,590,581,No,NoIntersection,LightRain,Other,...,0,0,0,0,0,0,0,0,0,0


In [32]:
# Train 데이터 셋 확인
train_final.head(3)

,AccidentId,Gravity,Date,Hour,Light,Department,Commune,InAgglomeration,IntersectionType,Weather,...,Maneuver_PassRight,Maneuver_Reverse,Maneuver_SameDirectionOrLane,Maneuver_Stopped,Maneuver_SwerveToLeft,Maneuver_SwerveToRight,Maneuver_TurnToLeft,Maneuver_TurnToRight,Maneuver_UTurnInLane,Maneuver_WrongWay
0,201800000001,NonLethal,2018-01-24,0 days 15:05:00,Daylight,590,5,No,Y-type,Normal,...,0,0,0,0,0,0,1,0,0,0
1,201800000002,NonLethal,2018-02-12,0 days 10:15:00,Daylight,590,11,Yes,Square,VeryGood,...,0,0,0,0,0,0,0,0,0,0
2,201800000003,NonLethal,2018-04-03,0 days 11:35:00,Daylight,590,477,Yes,T-type,Normal,...,0,0,0,0,0,0,1,0,0,0


In [33]:
# test_final 데이터 type 확인
pd.set_option("display.max_rows", None)
test_final.dtypes

AccidentId                                        int64
Date                                             object
Hour                                             object
Light                                            object
Department                                        int64
Commune                                           int64
InAgglomeration                                  object
IntersectionType                                 object
Weather                                          object
CollisionType                                    object
PostalAddress                                    object
GPSCode                                          object
Latitude                                        float64
Longitude                                       float64
RoadType                                         object
RoadNumber                                      float64
RoadSecNumber                                   float64
RoadLetter                                      

In [34]:
# train_final 데이터 type 확인
pd.set_option("display.max_rows", None)
train_final.dtypes

AccidentId                                        int64
Gravity                                          object
Date                                             object
Hour                                             object
Light                                            object
Department                                        int64
Commune                                           int64
InAgglomeration                                  object
IntersectionType                                 object
Weather                                          object
CollisionType                                    object
PostalAddress                                    object
GPSCode                                          object
Latitude                                        float64
Longitude                                       float64
RoadType                                         object
RoadNumber                                        int64
RoadSecNumber                                   

# 3. 데이터 저장하기

In [35]:
test_final.to_csv("./data/final_Feature/test_final.csv", index=False, encoding="utf-8-sig")
train_final.to_csv("./data/final_Feature/train_final.csv", index=False, encoding="utf-8-sig")